In [ ]:
import pycaret
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from pycaret.classification import setup, compare_models
from pycaret.classification import tune_model
from pycaret.classification import *
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix

from pycaret.classification import load_model

import pickle

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import recall_score
import pickle

Read Me

This file is used make prediction on small molecule dataset to predict their BBB permeability potential.
It uses 3 models to predict influx, efflux, PAMPA and BBB labels. Influx, Efflux BBB models use a subset of molecular descriptors (CDK, Chemopy, PaDel), while PAMPA models are trained with Chemopy fingerprints. 

Every efflux positive molecules are filtered out.
The following threshld are applied for the other 3 prediction to consider them positive:
    influx: prediction score >= 0.7
    PAMPA: prediction score >= 0.95
    BBB: prediction score >= 0.95

Composite score = influx score + PAMPA score + BBB score        Value range: 0-3
transport score = prediction score if it is > threshold, otherwise 0    



In [ ]:
#Define the models for each prediction

def define_models(t,f):

    if t == "influx":
        models =  ['XGBClassifier','RandomForestClassifier','GradientBoostingClassifier']

    elif t =="efflux":
        models = ['LGBMClassifier','ExtraTreesClassifier','RandomForestClassifier']

    elif t =="pampa":
        models = ['XGBClassifier','GradientBoostingClassifier','LogisticRegression']

    elif t =="bbb":
        models = ["LGBMClassifier","XGBClassifier","ExtraTreesClassifier"]

    return models

In [ ]:
#Load the trained classifier

def load_the_model(t,f,model_name):

    if f == "submd":
        f2 = "md"
    elif f =="subfp":
        f2 ="fp"

    model_file = f'../model_building/blood_brain_barrier/combined/models_sub_{f2}/{t}/combined_{f}_{t}_{model_name}_session_16_{f}'
    model = load_model(model_file)

    return model

In [ ]:
#Make prediction 

def make_prediction(model,df):
    predictions = predict_model(model, data=df, raw_score=True)
    
    return predictions

In [ ]:
#Load the unseen data (the files used here are not provided on Github - change it to the compound data with molecular fingerprints, and with moelcular descriptors)

df_submd = pd.read_feather('data/submd/small_mol_460k_submd.feather')
df_submd.replace([np.inf, -np.inf], 0, inplace=True) #Remove infinitie values




In [ ]:
#Create a dataset for only the prediction related information

def clean_df_for_info(df,):

    columns_to_keep = ['papyrus_SMILES','papyrus_inchi_key']
    df = df[columns_to_keep]

    return df

In [ ]:
#Calculate the average prediction score

def calculate_average(df):

    avg_values = df.iloc[:, [-6, -4, -2]].mean(axis=1)

    return avg_values

Prediction per transport

In [ ]:
#This is for influx, efflux and bbb

#define transport
t = 'efflux'    #transport - options: influx, efflux, bbb (DO NOT USE PAMPA!)
f = "submd"     #feature 

models = define_models(t,f)

df_info_influx = clean_df_for_info(df_submd)
print(f'Clean pred df: {len(df_info_influx)}')

#Iterate through models and make prediction
for m in models:
    classifier = load_the_model(t,f,m)
    predictions = make_prediction(classifier,df_submd)

    #Gather prediction and prediction score and add them to the prediction related dataframe
    pred_classes = predictions['prediction_label'].values
    probability_score= predictions['prediction_score_1'].values

    column_mod_score=f'{m}_pred_score_{t}_{f}'
    df_info_influx[column_mod_score] = probability_score

    column_mod_class = f'{m}_pred_class_{t}_{f}'
    df_info_influx[column_mod_class] = pred_classes

#calculate average pred.score
pred_score_means = calculate_average(df_info_influx)
column_score_mean = f'{t}_pred_score_final'
df_info_influx[column_score_mean] = pred_score_means

#Majority vote
counts = df_info_influx.iloc[:, [-6, -4, -2]].apply(lambda x: (x == 1).sum(), axis=1)
column_maj_class =f'status_{t}'
df_info_influx[column_maj_class] = counts.apply(lambda x: 1 if x > 1 else 0)

#Save prediction
file_name = f'bbb_pred_per_transport/{t}_prediction.feather'
df_info_influx.to_feather(file_name)


In [ ]:
#This if for PAMPA prediction only
#do not change the 't' and 'f' variable

t = "pampa"  #transport
f="subfp"   #feature

models = define_models(t)


df_info_pampa = pd.DataFrame()

for i in range(1,48):
    file_name = f'data/subfp/small_mols_papyrus_460k_chemopy_{i}.feather'
    df_tmp=pd.read_feather(file_name)

    df_info_pampa_tmp = clean_df_for_info(df_tmp)


    for m in models:
        classifier = load_the_model(t,f,m)
        predictions = make_prediction(classifier,df_tmp)

        pred_classes = predictions['prediction_label'].values
        probability_score= predictions['prediction_score_1'].values

        column_mod_score=f'{m}_pred_score_{t}_{f}'
        df_info_pampa_tmp[column_mod_score] = probability_score

        column_mod_class = f'{m}_pred_class_{t}_{f}'
        df_info_pampa_tmp[column_mod_class] = pred_classes


    #calculate average pred.score
    pred_score_means = calculate_average(df_info_pampa_tmp)
    column_score_mean = f'{t}_pred_score_final'
    df_info_pampa_tmp[column_score_mean] = pred_score_means

    #majority vote
    counts = df_info_pampa_tmp.iloc[:, [-6, -4, -2]].apply(lambda x: (x == 1).sum(), axis=1)
    column_maj_class =f'status_{t}'
    df_info_pampa_tmp[column_maj_class] = counts.apply(lambda x: 1 if x > 1 else 0)

    df_info_pampa = pd.concat([df_info_pampa,df_info_pampa_tmp],ignore_index=True)

#Save the prediction
df_info_pampa.to_feather("pred_per_transport/pampa_predictio.feather")

Further data manipulation, analysis

In [ ]:
#Load the predictions

influx_info = pd.read_feather('bbb_pred_per_transport/influx_prediction.feather')
influx_info = influx_info.reset_index(drop=True)

efflux_info = pd.read_feather('bbb_pred_per_transport/efflux_prediction.feather')
efflux_info = efflux_info.reset_index(drop=True)

pampa_info = pd.read_feather('bbb_pred_per_transport/pampa_prediction.feather')
pampa_info = pampa_info.reset_index(drop=True)

bbb_info = pd.read_feather('bbb_pred_per_transport/bbb_prediction.feather')
bbb_info = bbb_info.reset_index(drop=True)


In [ ]:
#Data check
# (efflux_info.papyrus_SMILES.sort_values().values == bbb_info.papyrus_SMILES.sort_values().values).all()

In [ ]:
#apply efflux filter: remove every efflux positive compound
#remove duplicates

data_noefflux = (pd.concat([influx_info.sort_values('papyrus_SMILES'), efflux_info.sort_values('papyrus_SMILES'), pampa_info.sort_values('papyrus_SMILES'), bbb_info.sort_values('papyrus_SMILES')], axis=1)
                   .drop_duplicates(subset="papyrus_inchi_key")
                   .query("status_efflux == 0")
                   )

In [ ]:
len(data_noefflux)

In [ ]:
#No efflux filter
#remove duplicates

data_withefflux = (pd.concat([influx_info.sort_values('papyrus_SMILES'), efflux_info.sort_values('papyrus_SMILES'), pampa_info.sort_values('papyrus_SMILES'), bbb_info.sort_values('papyrus_SMILES')], axis=1)
                   .drop_duplicates(subset="papyrus_inchi_key")
                   )

In [ ]:
#Define if from this point on which dataset you want to work with: after efflux filter, or without efflux filter
predictions = data_noefflux.reset_index(drop=True)

Composite Score Calculation

In [ ]:
#Define how the composite score is calculated, set thresholds

def calculate_comp_score(df):

    if df['influx_pred_score_final'] >= 0.7:
        influx = df['influx_pred_score_final'] * df['status_influx']
    else:
        influx = 0
    
    if df['pampa_pred_score_final'] >= 0.95:
        pampa = df['pampa_pred_score_final'] * df['status_pampa']
    else: 
        pampa =0

    if df['bbb_pred_score_final']  >= 0.95:
        bbb = df['bbb_pred_score_final'] * df['status_bbb']
    else:
        bbb = 0

    score = influx + pampa + bbb

    return  score

In [ ]:
#Calculate the composite score
predictions['composite_score'] = predictions.apply(calculate_comp_score, axis=1)

In [ ]:
#Sort molecules by the composite scre: descending

df_pred_sorted = predictions.sort_values(by='composite_score', ascending=False)
print(len(df_pred_sorted))

#Save the ranked prediction
df_pred_sorted.to_csv('efflux_neg_small_molecules_bbb_screening_ranked.csv', index=True)

In [ ]:
#Clean the dataset for better readability
#!!! This step removed predicted classes and prediction scores

columns_to_keep = ['papyrus_SMILES','papyrus_inchi_key','composite_score']
df_clean = predictions[columns_to_keep]

df_clean.to_csv("efflux_neg_small_molecules_bbb_screening_ranked_clean.csv", index=True)


In [ ]:
#Count BBB-permeability positive hits (threshold = 1.5)
#NOTE: if you have not applied the efflux filter previously, this might contain efflux positive compounds !!!

count_above_1_5 = df_clean[df_clean['composite_score'] >= 1.5].shape[0]

print(f"Number of hits with composite_score >= 1.5: {count_above_1_5}")



In [ ]:
final_positives = df_clean[df_clean['composite_score'] >= 1.5]

final_positives= final_positives.reset_index(drop=True)
final_positives.to_csv('small_mol_bbb_pos_15_efflux_neg.csv', index=True)